In [1]:
#pip install polygon-api-client requests python-dotenv pandas openai


In [2]:
# --- Core Libraries ---
import os
import time
import json
import warnings
from datetime import datetime, timedelta
from typing import Union, Iterator
from urllib3 import HTTPResponse
import requests
import hashlib

# --- Data Science & Math ---
import numpy as np
import pandas as pd
from scipy.stats import norm

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- API Clients ---
from polygon import RESTClient
from polygon.rest.models import UniversalSnapshot, SnapshotMarketType

# --- Google Colab Utilities ---
#from google.colab import userdata

# --- Progress Bar (Optional) ---
from tqdm.notebook import tqdm

# --- Warnings ---
warnings.filterwarnings("ignore")


In [3]:
# === ✅ CONFIG ===
tickers = [
    "META",  # Meta Platforms Inc.
    "AAPL",  # Apple Inc.
    "GME",   # GameStop Corp.
    "PLTR",  # Palantir Technologies Inc.
    "TSLA",  # Tesla Inc.
    "BABA",  # Alibaba Group Holding Ltd.
    "NVDA",  # NVIDIA Corporation
    "WW",    # WW International Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "SPOT",  # Spotify Technology S.A.
    "HOOD",  # Robinhood Markets Inc.
    "SFM",   # Sprouts Farmers Market Inc.
    "CHWY",  # Chewy Inc.
    "TASK",  # TaskUs Inc.
    "TEAM",  # Atlassian Corporation Plc
    "SGHC"   # Super Group (SGHC) Limited
]
# Combined list: Insider-driven value plays + macro/tech sensitivity
# tickers = [
#     # 🔐 Insider/Deep Value (Refined)
#     "NKE", "XOMA", "CRM", "JELD", "DOMO", "APA",
#     "SONO", "PBF", "VSCO", "ZYME", "AMAT", "ASAN",
#     "GME", "STAA", "EWTX", "SPY"

#     # 🧠 Tech Mega-Caps + Macro Trades
#     "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "SOXL"
# ]


TICKERS = [t.upper() for t in tickers if t.strip()]
tickers = TICKERS

TOP_N_PER_TICKER = 3
TOP_N_GLOBAL = 5
STRATEGY_OVERRIDE = None  # Optional, e.g., ["🧨 Gamma Spike"]
FORCE_LLM_REFRESH = False

my_contracts = [
    {"symbol": "ACHR270115C00020000", "fill_price": 0.65},
    {"symbol": "GME250718C00038000", "fill_price": 1.45},
]


In [4]:
TICKERS

['META',
 'AAPL',
 'GME',
 'PLTR',
 'TSLA',
 'BABA',
 'NVDA',
 'WW',
 'MSFT',
 'AMZN',
 'SPOT',
 'HOOD',
 'SFM',
 'CHWY',
 'TASK',
 'TEAM',
 'SGHC']

In [5]:
tickers

['META',
 'AAPL',
 'GME',
 'PLTR',
 'TSLA',
 'BABA',
 'NVDA',
 'WW',
 'MSFT',
 'AMZN',
 'SPOT',
 'HOOD',
 'SFM',
 'CHWY',
 'TASK',
 'TEAM',
 'SGHC']

In [6]:
# === Dynamic LLM Cache Key Based on Ticker Set ===
llm_ticker_hash = hashlib.md5(",".join(sorted(TICKERS)).encode()).hexdigest()[:8]
llm_cache_key = f"top_global_response_{llm_ticker_hash}"
FORCE_LLM_REFRESH = True  # ← flip to True to override


In [7]:
# # === Initialize and Enforce API Key Access ===

# # -- Load your Polygon key from Colab 'userdata' or env
# POLYGON_API_KEY = userdata.get("Polygon") or os.getenv("POLYGON_API_KEY")
# if not POLYGON_API_KEY:
#     raise ValueError("❌ Polygon API key not found. Set it via Colab 'userdata' or environment.")
# client = RESTClient(POLYGON_API_KEY)

# # -- Load your OpenRouter key the same way
# OPENROUTER_API_KEY = userdata.get("openrouter") or os.getenv("OPENROUTER_API_KEY")
# if not OPENROUTER_API_KEY:
#     raise ValueError("❌ OpenRouter API key not found. Set it via Colab 'userdata' or environment.")

# # ✅ Ensure get_llm_insights can find the key
# os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY


In [8]:
##MANUAL
POLYGON_API_KEY = "rqBRT4Bikq9MZKLtIKtuYFWxYrH7NvTU"
OPENROUTER_API_KEY = "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"

from polygon import RESTClient
client = RESTClient(POLYGON_API_KEY)

os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY


In [9]:
#print("From userdata:", userdata.get("Polygon"))


In [10]:
# ticker = "PLTR250425C00086000"
# underlying_symbol = ticker.split("2")[0]

# snapshot = client.get_snapshot_option(underlying_symbol, f"O:{ticker}")
# print(snapshot)

In [11]:
# ticker = "XRT250411C00070500"
# underlying_symbol = ticker.split("2")[0]

# snapshot = client.get_snapshot_option(underlying_symbol, f"O:{ticker}")
# print(snapshot)

In [12]:
underlying_asset = "AAPL"  # Define the asset
# === Option Chain Snapshot ===
url_snapshot = f"https://api.polygon.io/v3/snapshot/options/{underlying_asset}?apiKey={POLYGON_API_KEY}"
response_snapshot = requests.get(url_snapshot)
snapshot_data = response_snapshot.json()
print("✅ Snapshot retrieved")
print(snapshot_data["results"][0])  # Show first option contract

✅ Snapshot retrieved
{'day': {'change': -4.5, 'change_percent': -4.97, 'close': 86, 'high': 86, 'last_updated': 1746590400000000000, 'low': 85.65, 'open': 85.65, 'previous_close': 90.5, 'volume': 2, 'vwap': 85.825}, 'details': {'contract_type': 'call', 'exercise_style': 'american', 'expiration_date': '2025-05-09', 'shares_per_contract': 100, 'strike_price': 110, 'ticker': 'O:AAPL250509C00110000'}, 'greeks': {}, 'open_interest': 21, 'underlying_asset': {'ticker': 'AAPL'}}


In [13]:
# === Option Trades for a Specific Contract ===
option_ticker = 'O:AAPL230616C00150000'  # Replace with the one you're targeting
date = '2025-04-11'  # Historical trade date (make sure it's valid for your contract)

# Note: This endpoint is not `v3/trades/`, but rather part of `v2` or `v3/reference/options` endpoints depending on usage
url_trades = f"https://api.polygon.io/v3/trades/{option_ticker}?timestamp={date}&apiKey={POLYGON_API_KEY}"

response_trades = requests.get(url_trades)
trades = response_trades.json()

# Print sample trade
if "results" in trades:
    print("📈 First trade:", trades["results"][0])
else:
    print("❌ No trades found or error:", trades)

❌ No trades found or error: {'status': 'NOT_AUTHORIZED', 'request_id': '6269a8b277dc9c5ddf8ff1d9dc880520', 'message': 'You are not entitled to this data. Please upgrade your plan at https://polygon.io/pricing'}


In [14]:
option_ticker = 'O:AAPL250411C00100000'

url_option = f"https://api.polygon.io/v3/snapshot/options/{underlying_asset}/{option_ticker}?apiKey={POLYGON_API_KEY}"
response_option = requests.get(url_option)
option_snapshot = response_option.json()

# Inspect what comes back
from pprint import pprint
pprint(option_snapshot)


{'error': 'Options contract not found.',
 'request_id': 'e195c78a3a0e1da4200d1aaad0e522b6',
 'status': 'ERROR'}


In [15]:
# === Get Spot Price via Universal Snapshots ===
def get_spot_price(tickers: Union[str, list]) -> dict:
    try:
        if isinstance(tickers, str):
            tickers = [tickers]

        snapshots = client.list_universal_snapshots(ticker_any_of=tickers)
        results = {}

        for snap in snapshots:
            tkr = snap.ticker
            price = (
                getattr(snap.session, "price", None)
                or getattr(snap.session, "close", None)
                or getattr(snap.last_quote, "midpoint", None)
            )
            results[tkr] = float(price) if isinstance(price, (int, float)) else np.nan

        return results

    except Exception as e:
        print(f"❌ Failed to get spot price for {tickers}: {e}")
        return {t: np.nan for t in tickers}


In [16]:
def fetch_polygon_options(ticker: str, contract_ids: list[str]) -> pd.DataFrame:
    enriched_rows = []
    spot = get_spot_price(ticker)[ticker]

    for cid in contract_ids:
        try:
            snapshot = client.get_snapshot_option(ticker, f"O:{cid}")
            greeks = getattr(snapshot, "greeks", None)
            details = getattr(snapshot, "details", None)
            day = getattr(snapshot, "day", None)
            quote = getattr(snapshot, "last_quote", None)
            last_trade = getattr(snapshot, "last_trade", None)

            # Robust mid-price logic
            mid = next((
                val for val in [
                    getattr(quote, "midpoint", None),
                    getattr(snapshot, "fmv", None),
                    getattr(last_trade, "price", None),
                    (getattr(quote, "ask", None) + getattr(quote, "bid", None)) / 2
                    if getattr(quote, "ask", None) and getattr(quote, "bid", None)
                    else None
                ] if val is not None), None)

            price = next((
                val for val in [
                    getattr(day, "vwap", None),
                    getattr(day, "close", None),
                    getattr(snapshot, "fmv", None),
                    getattr(last_trade, "price", None),
                    mid
                ] if val is not None), None)

            cost = next(val for val in [getattr(quote, "ask", None), mid, price, getattr(quote, "bid", None)] if val)

            enriched = enrich_polygon_df(pd.DataFrame([{
                "symbol": details.ticker,
                "strike": details.strike_price,
                "expiration": details.expiration_date,
                "delta": greeks.delta,
                "theta": greeks.theta,
                "vega": greeks.vega,
                "gamma": greeks.gamma,
                "iv": snapshot.implied_volatility,
                "ask": quote.ask,
                "bid": quote.bid,
                "mid": mid,
                "price": price,
                "cost": cost,
                "open_interest": snapshot.open_interest,
                "volume": day.volume,
                "underlying": ticker
            }]), spot)

            enriched["price"] = spot
            enriched = assign_strategy_tags(enriched, spot)

            enriched_rows.append(enriched)

        except Exception as e:
            print(f"❌ Error fetching {cid}: {e}")

    return pd.concat(enriched_rows, ignore_index=True) if enriched_rows else pd.DataFrame()


In [17]:
# === Robust Option Chain Fetching ===
def fetch_option_chain_snapshots(ticker: str, contract_type: str = "call", limit: int = 30, expiry: str = None):
    """Fetch enriched option chain snapshot data from Polygon, with optional expiry filter."""
    options_data = []
    spot_price = get_spot_price(ticker).get(ticker, np.nan)

    try:
        contracts = client.list_snapshot_options_chain(
            ticker,
            params={
                "contract_type": contract_type,
                "order": "asc",
                "limit": limit,
                "sort": "ticker"
            }
        )

        for contract in contracts:
            # Filter by expiry if specified
            contract_exp = getattr(contract.details, "expiration_date", None)
            if expiry and contract_exp != expiry:
                continue

            greeks = getattr(contract, "greeks", None)
            day = getattr(contract, "day", None)
            quote = getattr(contract, "last_quote", None)

            # Extract quote pricing
            ask = getattr(quote, "ask", None) if quote else None
            bid = getattr(quote, "bid", None) if quote else None
            last_trade_price = getattr(contract.last_trade, "price", None) if getattr(contract, "last_trade", None) else None
            day_close = getattr(day, "close", None)
            day_vwap = getattr(day, "vwap", None)

            # Determine 'mid'
            mid = (
                (ask + bid) / 2 if ask is not None and bid is not None
                else last_trade_price
                or day_vwap
                or day_close
            )

            # === Cost logic with priority ===
            if last_trade_price is not None:
                cost = last_trade_price
                cost_source = "last_trade"
            elif ask is not None and bid is not None:
                cost = (ask + bid) / 2
                cost_source = "mid"
            elif ask is not None:
                cost = ask
                cost_source = "ask"
            elif bid is not None:
                cost = bid
                cost_source = "bid"
            else:
                cost = day_close or day_vwap
                cost_source = "fallback"

            options_data.append({
                "symbol": getattr(contract.details, "ticker", np.nan),
                "strike": getattr(contract.details, "strike_price", np.nan),
                "expiration": contract_exp,
                "delta": getattr(greeks, "delta", np.nan),
                "theta": getattr(greeks, "theta", np.nan),
                "vega": getattr(greeks, "vega", np.nan),
                "gamma": getattr(greeks, "gamma", np.nan),
                "iv": getattr(contract, "implied_volatility", np.nan),
                "ask": ask,
                "bid": bid,
                "mid": mid,
                "price": last_trade_price or day_close or day_vwap,
                "cost": cost,
                "cost_source": cost_source,
                "open_interest": getattr(contract, "open_interest", np.nan),
                "volume": getattr(day, "volume", None),
                "underlying": ticker
            })

    except Exception as e:
        print(f"❌ Error fetching option chain for {ticker}: {e}")

    return pd.DataFrame(options_data), float(spot_price) if pd.notna(spot_price) else np.nan


In [18]:
# # === Robust Option Chain Fetching ===
# def fetch_option_chain_snapshots(ticker: str, contract_type: str = "call", limit: int = 30):
#     """Fetch enriched option chain snapshot data from Polygon."""
#     options_data = []
#     spot_price = get_spot_price(ticker).get(ticker, np.nan)

#     try:
#         contracts = client.list_snapshot_options_chain(
#             ticker,
#             params={
#                 "contract_type": contract_type,
#                 "order": "asc",
#                 "limit": limit,
#                 "sort": "ticker"
#             }
#         )

#         for contract in contracts:
#             greeks = getattr(contract, "greeks", None)
#             day = getattr(contract, "day", None)
#             quote = getattr(contract, "last_quote", None)

#             # Extract quote pricing
#             ask = getattr(quote, "ask", None) if quote else None
#             bid = getattr(quote, "bid", None) if quote else None
#             last_trade_price = getattr(contract.last_trade, "price", None) if getattr(contract, "last_trade", None) else None

#             # Extract fallback pricing from day snapshot
#             day_close = getattr(day, "close", None)
#             day_vwap = getattr(day, "vwap", None)

#             # Robust mid logic
#             mid = (
#                 (ask + bid) / 2 if ask is not None and bid is not None
#                 else last_trade_price
#                 or day_vwap
#                 or day_close
#             )

#             # Cost fallback logic
#             cost = ask or mid or day_vwap or day_close or bid

#             options_data.append({
#                 "symbol": getattr(contract.details, "ticker", np.nan),
#                 "strike": getattr(contract.details, "strike_price", np.nan),
#                 "expiration": getattr(contract.details, "expiration_date", np.nan),
#                 "delta": getattr(greeks, "delta", np.nan),
#                 "theta": getattr(greeks, "theta", np.nan),
#                 "vega": getattr(greeks, "vega", np.nan),
#                 "gamma": getattr(greeks, "gamma", np.nan),
#                 "iv": getattr(contract, "implied_volatility", np.nan),
#                 "ask": ask,
#                 "bid": bid,
#                 "mid": mid,
#                 "price": last_trade_price or day_close or day_vwap,
#                 "cost": cost,
#                 "open_interest": getattr(contract, "open_interest", np.nan),
#                 "volume": getattr(day, "volume", None),
#                 "underlying": ticker
#             })

#     except Exception as e:
#         print(f"❌ Error fetching option chain for {ticker}: {e}")

#     return pd.DataFrame(options_data), float(spot_price) if pd.notna(spot_price) else np.nan


In [19]:
# === Composite Score Function ===
def compute_composite_score(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Handle any missing inputs gracefully
    df["expected_roi"] = pd.to_numeric(df.get("expected_roi", 0), errors="coerce").fillna(0)
    df["mc_winrate"] = pd.to_numeric(df.get("mc_winrate", 0), errors="coerce").fillna(0)
    df["prob_hit_target"] = pd.to_numeric(df.get("prob_hit_target", 0), errors="coerce").fillna(0)
    df["theta_decay"] = pd.to_numeric(df.get("theta_decay", 0), errors="coerce").fillna(0)

    # Composite scoring logic: reward profit potential, penalize decay
    df["composite_score"] = (
        df["expected_roi"]
        + df["mc_winrate"] * 80
        + df["prob_hit_target"] * 20
        - df["theta_decay"]
    )

    return df


In [20]:
def fetch_manual_contracts(contract_info: list[dict]) -> pd.DataFrame:
    enriched_rows = []

    symbols = [c["symbol"] for c in contract_info]
    base_tickers = list({cid[:cid.index("2")] for cid in symbols})
    spot_prices = get_spot_price(base_tickers)

    fill_price_map = {c["symbol"]: c.get("fill_price", np.nan) for c in contract_info}

    for cid in symbols:
        try:
            underlying_symbol = cid.split("2")[0]
            snapshot = client.get_snapshot_option(underlying_symbol, f"O:{cid}")
            greeks = getattr(snapshot, "greeks", None)
            details = getattr(snapshot, "details", None)
            day = getattr(snapshot, "day", None)
            quote = getattr(snapshot, "last_quote", None)
            last_trade = getattr(snapshot, "last_trade", None)

            ask = getattr(quote, "ask", None) if quote else None
            bid = getattr(quote, "bid", None) if quote else None
            midpoint = getattr(quote, "midpoint", None) if quote else None

            fmv = getattr(snapshot, "fmv", None)
            last_trade_price = getattr(last_trade, "price", None) if last_trade else None
            manual_mid = (ask + bid) / 2 if ask is not None and bid is not None else None

            day_vwap = getattr(day, "vwap", None) if day else None
            day_close = getattr(day, "close", None) if day else None
            day_open = getattr(day, "open", None) if day else None
            day_high = getattr(day, "high", None) if day else None
            day_low = getattr(day, "low", None) if day else None

            mid = next((v for v in [
                midpoint, fmv, last_trade_price, manual_mid,
                day_close, day_vwap, day_open, day_high, day_low
            ] if v is not None), np.nan)

            price = mid
            cost = next((v for v in [ask, mid, bid, price] if v is not None), np.nan)

            spot = spot_prices.get(underlying_symbol, np.nan)
            if np.isnan(spot):
                print(f"⚠️ Spot price missing for {cid} — skipping.")
                continue

            row_data = {
                "symbol": getattr(details, "ticker", None),
                "strike": getattr(details, "strike_price", None),
                "expiration": getattr(details, "expiration_date", None),
                "delta": getattr(greeks, "delta", None),
                "theta": getattr(greeks, "theta", None),
                "vega": getattr(greeks, "vega", None),
                "gamma": getattr(greeks, "gamma", None),
                "iv": getattr(snapshot, "implied_volatility", None),
                "ask": ask,
                "bid": bid,
                "mid": mid,
                "price": price,
                "cost": cost,
                "open_interest": getattr(snapshot, "open_interest", None),
                "volume": getattr(day, "volume", None) if day else np.nan,
                "underlying": underlying_symbol,
                "fill_price": fill_price_map.get(cid, np.nan),
                "spot_price": spot,
                "underlying_price": spot
            }

            enriched = enrich_polygon_df(pd.DataFrame([row_data]), spot)
            enriched = assign_strategy_tags(enriched, spot)
            enriched = compute_composite_score(enriched)

            # Add breakeven for LLM compatibility
            enriched["breakeven_price"] = enriched["strike"] + enriched["cost"]

            # Calculate P/L
            enriched["fill_price"] = pd.to_numeric(enriched.get("fill_price"), errors="coerce")
            enriched["mid"] = pd.to_numeric(enriched.get("mid"), errors="coerce")

            enriched["unrealized_pnl"] = np.where(
                enriched["mid"].notnull() & enriched["fill_price"].notnull(),
                (enriched["mid"] - enriched["fill_price"]) * 100,
                np.nan
            )

            enriched["pnl_pct"] = np.where(
                enriched["mid"].notnull() & enriched["fill_price"].notnull() & (enriched["fill_price"] != 0),
                ((enriched["mid"] - enriched["fill_price"]) / enriched["fill_price"]) * 100,
                np.nan
            )

            # Add enrichment flag
            enriched["valid_enrichment"] = enriched[
                ["expected_roi", "mc_winrate", "prob_ITM"]
            ].notnull().all(axis=1)

            # Optional: visually tag manual entries
            enriched["smart_tags"] = enriched["smart_tags"].astype(str) + " 🛠️ Manual"

            enriched_rows.append(enriched)

        except Exception as e:
            print(f"❌ Error fetching {cid}: {e}")

    return pd.concat(enriched_rows, ignore_index=True) if enriched_rows else pd.DataFrame()


In [21]:
# # === Enrichment Helper for Manual Contracts ===
# def enrich_manual_contracts(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()

#     if df.empty or "strike" not in df.columns:
#         print("⚠️ Skipping enrichment: DataFrame missing or incomplete.")
#         return pd.DataFrame()

#     # Set underlying price from spot_price if available
#     if "underlying_price" not in df.columns or df["underlying_price"].isnull().any():
#         spot_mean = df["spot_price"].mean() if "spot_price" in df.columns else np.nan
#         df["underlying_price"] = spot_mean

#     # Run enrichment only if metrics are missing
#     if "expected_roi" not in df.columns or df["expected_roi"].isna().all():
#         print("🔁 Enriching manual contracts (missing ROI)...")
#         df = enrich_polygon_df(df, spot_price=df["underlying_price"].mean())
#         df = assign_strategy_tags(df, spot_price=df["underlying_price"].mean())

#     # Score and tag (if needed)
#     if "composite_score" not in df.columns:
#         df = compute_composite_score(df)

#     # P/L Calculations
#     df["fill_price"] = pd.to_numeric(df.get("fill_price"), errors="coerce")
#     df["mid"] = pd.to_numeric(df.get("mid"), errors="coerce")

#     df["unrealized_pnl"] = np.where(
#         df["mid"].notnull() & df["fill_price"].notnull(),
#         (df["mid"] - df["fill_price"]) * 100,
#         np.nan
#     )

#     df["pnl_pct"] = np.where(
#         df["mid"].notnull() & df["fill_price"].notnull() & (df["fill_price"] != 0),
#         ((df["mid"] - df["fill_price"]) / df["fill_price"]) * 100,
#         np.nan
#     )

#     # Flag enrichment validity
#     df["valid_enrichment"] = df[["expected_roi", "mc_winrate", "prob_ITM"]].notnull().all(axis=1)

#     return df


In [22]:
# === Robust Option Enrichment Function ===
def enrich_polygon_df(df: pd.DataFrame, spot_price: float) -> pd.DataFrame:
    df = df.copy()

    required_cols = ["strike", "expiration"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column '{col}' in option DataFrame.")

    numeric_cols = ["strike", "cost", "mid", "ask", "bid"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Fill missing bid/ask if needed
    if "bid" not in df.columns:
        df["bid"] = np.nan
    if "ask" not in df.columns:
        df["ask"] = np.nan

    # Calculate mid if not already
    if "mid" not in df.columns:
        df["mid"] = (df["ask"] + df["bid"]) / 2
    else:
        df["mid"] = df["mid"].fillna((df["ask"] + df["bid"]) / 2)

    # Fallback price placeholder
    if "price" not in df.columns:
        df["price"] = np.nan

    # Estimate fill price using vwap or close
    df["fill_price_est"] = df.apply(estimate_fill_price, axis=1)

    # Use estimated fill price as cost (with safe fallback)
    df["cost"] = df["price"].fillna(df["fill_price_est"]).fillna(df["mid"]).fillna(df["ask"]).fillna(df["price"]).fillna(df["bid"])

    # Dates and Days to expiry
    df["expiration"] = pd.to_datetime(df["expiration"], errors="coerce")
    df["days_to_expiry"] = (df["expiration"] - pd.to_datetime(datetime.today().date())).dt.days

    # Underlying spot price
    df["underlying_price"] = spot_price

    # Breakeven & ROI
    df["breakeven"] = df["strike"] + df["price"]
    df["roi"] = ((df["breakeven"] - spot_price) / spot_price) * 100

    # Midfill profit/value
    midfill = df["mid"].fillna(df["price"])
    df["midfill_profit"] = ((df["strike"] - spot_price) + midfill).where(spot_price < df["strike"], 0)
    df["value_score"] = df["midfill_profit"] / df["price"].replace(0, np.nan)

    # Probability in the money
    df["prob_ITM"] = df.apply(
        lambda row: 1 if spot_price >= row["strike"]
        else np.exp(-abs(row["strike"] - spot_price) / (spot_price * 0.15)),
        axis=1
    )

    # Target price modeling
    fib_ratio = 1.618
    fib_component = (df["strike"] - spot_price).clip(upper=spot_price * 1.5) * fib_ratio
    fib_target = df["strike"] + fib_component
    naive_target = df["strike"] + (df["price"] * 1.5)
    deep_otm = (df["strike"] - spot_price) > (spot_price * 0.25)
    weight = np.where(deep_otm, 0.3, 0.5)
    df["target_price"] = weight * fib_target + (1 - weight) * naive_target

    # Simulate target price winrate and ROI
    df[["target_price", "mc_winrate", "expected_roi", "prob_hit_target"]] = df.apply(
        simulate_target_price, axis=1
    )

    # Theta decay and signal scoring
    df["theta_decay"] = df.get("theta", 0).fillna(0).astype(float) * df["days_to_expiry"]
    df["signal_score"] = (
        df["expected_roi"].fillna(0) +
        df["mc_winrate"].fillna(0) * 80 +
        df["prob_hit_target"].fillna(0) * 20 -
        df["theta_decay"].fillna(0)
    )

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

In [23]:
# def simulate_target_price(row):
#     try:
#         spot = row["underlying_price"]
#         iv = row["iv"]
#         T = row["days_to_expiry"] / 365
#         cost = row["cost"]
#         strike = row["strike"]
#         target_price = row.get("target_price", None)

#         if pd.isna(spot) or pd.isna(iv) or pd.isna(T) or T <= 0 or pd.isna(cost):
#             return pd.Series([np.nan, np.nan, np.nan, np.nan])

#         simulations = 1000
#         simulated_prices = np.random.normal(
#             loc=spot,
#             scale=spot * iv * np.sqrt(T),
#             size=simulations
#         )

#         final_payoffs = np.maximum(simulated_prices - strike, 0)
#         win_rate = np.mean(final_payoffs > cost)
#         expected_profit = np.mean(final_payoffs) - cost
#         expected_roi = expected_profit / cost * 100 if cost > 0 else np.nan

#         prob_hit_target = (
#             np.mean(simulated_prices >= target_price)
#             if target_price and not pd.isna(target_price)
#             else np.nan
#         )

#         return pd.Series([target_price, win_rate, expected_roi, prob_hit_target])
#     except Exception as e:
#         print(f"❌ Simulation error: {e}")
#         return pd.Series([np.nan, np.nan, np.nan, np.nan])


In [24]:
def simulate_target_price(row):
    try:
        spot = row["underlying_price"]
        iv = row["iv"]
        T = row["days_to_expiry"] / 365
        cost = row["price"]
        strike = row["strike"]
        target_price = row.get("target_price", np.nan)

        if pd.isna(spot) or pd.isna(iv) or pd.isna(T) or T <= 0 or pd.isna(cost):
            return pd.Series([np.nan, np.nan, np.nan, np.nan])

        # Default to breakeven + 10% if no target provided
        if pd.isna(target_price):
            target_price = strike + cost * 1.1

        simulations = 1000
        simulated_prices = np.random.normal(
            loc=spot,
            scale=spot * iv * np.sqrt(T),
            size=simulations
        )

        final_payoffs = np.maximum(simulated_prices - strike, 0)
        win_rate = np.mean(final_payoffs > cost)
        expected_profit = np.mean(final_payoffs) - cost
        expected_roi = expected_profit / cost * 100 if cost > 0 else np.nan
        prob_hit_target = np.mean(simulated_prices >= target_price)

        return pd.Series([target_price, win_rate, expected_roi, prob_hit_target])
    except Exception as e:
        print(f"❌ Simulation error: {e}")
        return pd.Series([np.nan, np.nan, np.nan, np.nan])


In [25]:
def get_max_pain(ticker_str, expiry_date, spot_dict):
    spot = spot_dict.get(ticker_str, np.nan)
    if pd.isna(spot):
        print(f"❌ No spot price available for {ticker_str}.")
        return None

    print(f"Spot for {ticker_str}: {spot}")
    print(f"Expiry date for {ticker_str}: {expiry_date}")

    try:
        # ✅ Fetch all, then filter by expiry
        df_calls, _ = fetch_option_chain_snapshots(ticker_str, contract_type="call", limit=100)
        df_puts, _ = fetch_option_chain_snapshots(ticker_str, contract_type="put", limit=100)

        # Normalize expiration formatting
        df_calls["expiration"] = df_calls["expiration"].astype(str).str.strip()
        df_puts["expiration"] = df_puts["expiration"].astype(str).str.strip()
        expiry_date = str(expiry_date).strip()

        # Filter down to desired expiry
        df_calls = df_calls[df_calls["expiration"] == expiry_date]
        df_puts = df_puts[df_puts["expiration"] == expiry_date]

        if df_calls.empty or df_puts.empty:
            print(f"⚠ No options for {ticker_str} on {expiry_date}.")
            return None

        df = pd.concat([df_calls, df_puts])
        strikes = sorted(set(df['strike']))
        pain_df = pd.DataFrame({'strike': strikes})

        # Calculate call and put losses at each strike
        pain_df['call_loss'] = pain_df['strike'].apply(
            lambda x: sum(((x - df_calls['strike']).clip(lower=0)) * df_calls['open_interest'])
        )
        pain_df['put_loss'] = pain_df['strike'].apply(
            lambda x: sum(((df_puts['strike'] - x).clip(lower=0)) * df_puts['open_interest'])
        )
        pain_df['total_loss'] = pain_df['call_loss'] + pain_df['put_loss']

        # Max Pain is the strike with lowest total loss
        max_pain_strike = pain_df.loc[pain_df['total_loss'].idxmin(), 'strike']

        return max_pain_strike

    except Exception as e:
        print(f"❌ Error processing {ticker_str}: {e}")
        return None


In [26]:
def prob_itm(row):
    S = row['spot']
    K = row['strike']
    T = row['days_left'] / 365
    sigma = row['iv']
    if T <= 0 or sigma <= 0:
        return 0
    d2 = (np.log(S / K) + (sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    return round(norm.cdf(d2), 2) if row['option_type'] == 'call' else round(1 - norm.cdf(d2), 2)


In [27]:
def plot_option_summary(df):
    plt.figure(figsize=(10,5))

    # Scatter plot for implied volatility vs. probability ITM
    plt.scatter(df['prob_ITM'], df['iv'], c=df['value_score'], s=100, alpha=0.5, edgecolors='k', cmap='viridis')
    plt.colorbar(label="Value Score")
    plt.title("IV vs. Probability of ITM (bubble size = value score)")
    plt.xlabel("Probability ITM")
    plt.ylabel("Implied Volatility")
    plt.grid(True)
    plt.show()

    # Max Pain vs. Signal Score (from enrichment)
    plt.figure(figsize=(10,5))
    plt.scatter(df['max_pain'], df['signal_score'], c='b', alpha=0.7)
    plt.title("Max Pain vs. Signal Score")
    plt.xlabel("Max Pain")
    plt.ylabel("Signal Score")
    plt.grid(True)
    plt.show()


In [28]:
# === Visualization Toolkit ===
def plot_roi_distribution(df: pd.DataFrame):
    plt.figure(figsize=(10, 5))
    sns.histplot(df["expected_roi"], bins=50, kde=True, color="skyblue")
    plt.axvline(df["expected_roi"].mean(), color="green", linestyle="--", label="Mean ROI")
    plt.axvline(df["expected_roi"].median(), color="orange", linestyle=":", label="Median ROI")
    plt.title("Expected ROI Distribution")
    plt.xlabel("Expected ROI (%)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_breakeven_vs_target(df: pd.DataFrame):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x=df["breakeven_price"],
        y=df["target_price"],
        hue=df["composite_score"],
        size=df["expected_roi"],
        palette="coolwarm",
        sizes=(20, 200),
        alpha=0.7,
        edgecolor="black"
    )
    plt.plot([df["breakeven_price"].min(), df["breakeven_price"].max()],
             [df["breakeven_price"].min(), df["breakeven_price"].max()],
             color="gray", linestyle="--", label="Breakeven = Target")
    plt.title("Target Price vs. Breakeven Price")
    plt.xlabel("Breakeven Price")
    plt.ylabel("Target Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_composite_heatmap(df: pd.DataFrame):
    pivot = df.pivot_table(index="days_to_expiry", columns="delta", values="composite_score", aggfunc="mean")
    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot, cmap="YlGnBu", linewidths=0.1, linecolor="gray")
    plt.title("Composite Score Heatmap (Delta vs DTE)")
    plt.xlabel("Delta")
    plt.ylabel("Days to Expiry")
    plt.tight_layout()
    plt.show()


In [29]:
# === Strategy Tagging Engine ===
def assign_strategy_tags(df: pd.DataFrame, spot_price: float = None) -> pd.DataFrame:
    tags = []

    for _, row in df.iterrows():
        row_tags = []

        # Classic setups
        if row.get("gamma", 0) > 0.02 and row.get("cost", 10) < 1.5 and row.get("days_to_expiry", 999) <= 14:
            row_tags.append("🧨 Gamma Spike")
        if 0.45 < row.get("delta", 0) < 0.55 and row.get("iv", 0) > 0.4:
            row_tags.append("🎯 Straddle Candidate")
        if row.get("delta", 0) > 0.85 and row.get("strike", 0) < spot_price * 0.9:
            row_tags.append("🏦 Synthetic Long")
        if row.get("theta", 0) > 0 and row.get("open_interest", 0) > 1000:
            row_tags.append("🌾 Theta Farm")
        if row.get("delta", 1) < 0.2 and row.get("cost", 10) < 0.5:
            row_tags.append("🎲 Micro Bet")

        # Max Pain / Price alignment
        if row.get("below_max_pain") and row.get("within_10pct"):
            row_tags.append("📈 Below Max Pain")
        if abs(row.get("strike", 0) - row.get("max_pain", 0)) / max(row.get("max_pain", 1), 1) < 0.05:
            row_tags.append("🧲 Breakeven Magnet")
        if spot_price and 0 < (row.get("max_pain", 0) - spot_price) / spot_price < 0.1:
            row_tags.append("🏹 Rebound Play")

        # NEW: Max Pain Revert
        if (
            row.get("strike", 0) < row.get("spot_price", 0)
            and row.get("max_pain", 0) > row.get("strike", 0)
            and row.get("days_to_expiry", 999) <= 21
        ):
            row_tags.append("📉 Max Pain Revert")

        # Risk/Reward overlay
        if row.get("probabilistic_risk"):
            row_tags.append("⚠️ Probabilistic Risk")
        if row.get("conviction_flag"):
            row_tags.append("✅ Conviction Grade")
        if row.get("kelly_weight", 0) > 0.25:
            row_tags.append("📊 Kelly-Backed")
        if row.get("expected_roi", 0) > 300:
            row_tags.append("💸 ROI Rich")
        if row.get("expected_roi", 0) < -50:
            row_tags.append("⚠️ ROI Risk")

        # Temporal
        days = row.get("days_to_expiry", 99)
        if days > 180 and row.get("delta", 0) > 0.5:
            row_tags.append("📅 Core Hold")
        elif 21 <= days <= 60:
            row_tags.append("🕰 Swing Candidate")
        elif days < 21:
            row_tags.append("🚀 Short-Term Play")

        # Speculative tag
        if row.get("delta", 0) < 0.25 and row.get("expected_roi", 0) > 200:
            row_tags.append("🎲 Speculative Longshot")

        tags.append(", ".join(row_tags) if row_tags else "—")

    df["smart_tags"] = tags
    return df


In [30]:
def estimate_fill_price(row, default_to_close=True):
    """
    Estimate a realistic fill price for an option using Polygon Starter plan data.

    Parameters:
    - row: A DataFrame row with 'vwap', 'close', 'volume', 'open_interest'
    - default_to_close: Whether to fallback to close price if vwap is missing

    Returns:
    - float: estimated fill price
    """
    vwap = row.get("vwap")
    close = row.get("close")
    volume = row.get("volume", 0)
    oi = row.get("open_interest", 0)

    if pd.notna(vwap) and volume > 0:
        return vwap  # Best estimate based on trade-weighted price
    elif pd.notna(close) and default_to_close:
        return close  # Less accurate fallback
    else:
        return np.nan  # No basis to estimate


In [31]:
def fetch_polygon_chain(tickers: list = None) -> pd.DataFrame:
    """Fetch and enrich snapshot data for a list of tickers."""
    tickers = tickers or DEFAULT_TICKERS
    spot_dict = get_spot_price(tickers)
    all_snapshots = []

    for t in tickers:
        df, _ = fetch_option_chain_snapshots(t, contract_type="call", limit=30)
        spot = spot_dict.get(t, np.nan)

        if df.empty or pd.isna(spot):
            continue

        expiry_date = df["expiration"].min()
        max_pain_strike = get_max_pain(t, expiry_date, spot_dict)

        enriched = enrich_polygon_df(df, spot)
        enriched["max_pain"] = max_pain_strike
        enriched["distance_to_max_pain"] = abs(enriched["strike"] - max_pain_strike) / spot
        all_snapshots.append(enriched)

    return pd.concat(all_snapshots, ignore_index=True) if all_snapshots else pd.DataFrame()


In [32]:
def select_top_options_by_ticker(df: pd.DataFrame, top_n: int = 3) -> pd.DataFrame:
    if df.empty or "underlying" not in df.columns:
        print("⚠ DataFrame missing or improperly enriched.")
        return df

    df = df.copy()

    # ✅ Optional enrichment validity filter
    if "valid_enrichment" in df.columns:
        df = df[df["valid_enrichment"]]

    # ✅ Core filters
    df = df[
        (df["days_to_expiry"] > 0) &
        (df["expected_roi"].notna()) &
        (df["mc_winrate"].notna()) &
        (df["open_interest"].fillna(0) >= 50) &
        (df["volume"].fillna(0) >= 10)
    ]

    # ✅ Sanity filters
    df = df[
        (df["cost"] > 0.05) &
        (df["midfill_profit"].notna()) &
        (df["value_score"].notna()) &
        (df["value_score"] > 0.5)
    ]

    if df.empty:
        print("⚠ No contracts passed the filtering criteria.")
        return df

    # ✅ Rank top N by ticker using signal_score
    df["rank"] = df.groupby("underlying")["signal_score"].rank(method="first", ascending=False)
    top_df = df[df["rank"] <= top_n].sort_values(["underlying", "rank"]).drop(columns="rank")

    print(f"✅ Selected top {top_n} liquid and valid options per ticker.")
    return top_df


In [33]:

# Define all the enriched option columns with deeper logic
column_definitions = [
    {"Column": "symbol", "Formula": "From contract details", "Description": "Full option contract ticker (e.g., O:AAPL250118C00125000)"},
    {"Column": "strike", "Formula": "From contract details", "Description": "Strike price of the option"},
    {"Column": "expiration", "Formula": "From contract details", "Description": "Option expiration date"},
    {"Column": "delta", "Formula": "From greeks", "Description": "Rate of change in option price per $1 move in underlying"},
    {"Column": "theta", "Formula": "From greeks", "Description": "Time decay of the option per calendar day"},
    {"Column": "vega", "Formula": "From greeks", "Description": "Sensitivity to 1% change in implied volatility"},
    {"Column": "gamma", "Formula": "From greeks", "Description": "Rate of change of delta as the underlying price changes"},
    {"Column": "iv", "Formula": "From snapshot", "Description": "Implied volatility (annualized %)"},
    {"Column": "ask", "Formula": "From quote", "Description": "Ask price (what buyers are offering)"},
    {"Column": "bid", "Formula": "From quote", "Description": "Bid price (what sellers are asking)"},
    {"Column": "mid", "Formula": "(ask + bid) / 2", "Description": "Midpoint between bid and ask prices"},
    {"Column": "price", "Formula": "vwap → close → mid", "Description": "Fallback estimate of last traded price"},
    {"Column": "cost", "Formula": "ask → mid → price → bid", "Description": "Estimated option cost at entry"},
    {"Column": "open_interest", "Formula": "From snapshot", "Description": "Outstanding contracts not yet closed"},
    {"Column": "volume", "Formula": "From snapshot", "Description": "Contracts traded during the current session"},
    {"Column": "underlying", "Formula": "Parsed from ticker", "Description": "Stock symbol of the underlying asset"},
    {"Column": "days_to_expiry", "Formula": "expiration − today", "Description": "Number of days remaining to expiration"},
    {"Column": "underlying_price", "Formula": "From universal snapshot", "Description": "Spot price of the underlying stock"},
    {"Column": "breakeven", "Formula": "strike + cost", "Description": "Minimum underlying price needed to break even"},
    {"Column": "roi", "Formula": "((breakeven − spot) / spot) × 100", "Description": "Projected raw return to reach breakeven"},
    {"Column": "midfill_profit", "Formula": "((strike − spot) + mid) if OTM", "Description": "Estimated profit if midpoint is filled and contract ends ITM"},
    {"Column": "value_score", "Formula": "midfill_profit / cost", "Description": "Heuristic for return per dollar invested"},
    {"Column": "prob_ITM", "Formula": "exp(−|strike − spot| / (spot × 0.15))", "Description": "Quick estimate of probability finishing in the money"},
    {"Column": "target_price", "Formula": "weighted(fib, naive)", "Description": "Projected target using a weighted blend of Fib extension and naive method"},
    {"Column": "mc_winrate", "Formula": "Monte Carlo simulation", "Description": "Fraction of simulations where the trade ends profitable"},
    {"Column": "expected_roi", "Formula": "Avg ROI from winning paths", "Description": "Simulated average return from profitable paths only"},
    {"Column": "prob_hit_target", "Formula": "% MC paths hit target", "Description": "Share of simulations reaching the target price"},
    {"Column": "theta_decay", "Formula": "theta × days_to_expiry", "Description": "Cumulative value lost from theta decay"},
    {"Column": "signal_score", "Formula": "expected_roi + (winrate × 80) + (hit × 20) − theta_decay", "Description": "Composite score balancing profit, probability, and decay"},
    {"Column": "valid_enrichment", "Formula": "~isna(expected_roi or mc_winrate)", "Description": "Flag for filtering out incomplete simulations"},
    {"Column": "smart_tags", "Formula": "Based on gamma, delta, theta, IV, cost, DTE, OI", "Description": "Categorical labels indicating strategy fit (e.g., 🧨 Gamma Spike, 🏦 Synthetic Long, 🎯 Straddle Candidate)"}
]


df_definitions = pd.DataFrame(column_definitions)

dictionary_prompt = "### Column Definitions:\n" + "\n".join(
    f"• {row['Column']} = {row['Formula']} → {row['Description']}"
    for _, row in df_definitions.iterrows()
)



In [34]:
# def analyze_openrouter_trades(df: pd.DataFrame, top_n: int = 5, context_note: str = "Top enriched option plays") -> str:
#     intro = f"""
# You are an advanced options trading assistant. Evaluate each of these scouted option plays based on:

# - Expected ROI, winrate, and probability of ITM
# - Strategic metrics: IV, delta, theta, value score
# - Breakeven realism and gamma/volatility factors
# - Tag-based signals (e.g., Gamma Spike, Conviction Grade)
# - Make an overall assessment: Is this worth considering for entry?

# {context_note.strip()}

# --- Begin Trade Evaluation ---
# """.strip()

#     sorted_df = df.sort_values("composite_score", ascending=False).head(top_n)
#     trade_blocks = []

#     for _, row in sorted_df.iterrows():
#         block = f"""
# ---
# 🔹 **Symbol:** {row['symbol']}
# - Underlying: {row['underlying']} @ ${row['underlying_price']:.2f} | Strike: {row['strike']} | Expiry: {row['expiration']}
# - Cost: ${row['cost']:.2f} | Mid: {row.get('mid', '—')}
# - Breakeven: ${row.get('breakeven_price', 0):.2f} | Target: ${row.get('target_price', 0):.2f}
# - Days to Expiry: {row.get('days_to_expiry', '—')} | Expected ROI: {row.get('expected_roi', 0):.2f}% | Winrate: {row.get('mc_winrate', 0):.2f} | Prob ITM: {row.get('prob_ITM', 0):.2f}
# - Greeks → Delta: {row.get('delta', 0):.2f} | Theta: {row.get('theta', 0):.2f} | IV: {row.get('iv', 0):.2f}
# - Tags: {row.get('smart_tags', '—')}

# 📈 Signal Score: {row.get('signal_score', 0):.2f} | Composite Score: {row.get('composite_score', 0):.2f}
# """.strip()

#         trade_blocks.append(block)

#     # Highlight best pick
#     best = sorted_df.iloc[0]
#     best_summary = f"""
# 🎯 **Best Opportunity:** {best['symbol']} — Score: {best['composite_score']:.2f}, Tags: {best.get('smart_tags', '—')}
# Rationale: High expected ROI, strong scoring, and strategic tag alignment. Worth further attention.
# """.strip()

#     return intro + "\n\n" + "\n\n".join(trade_blocks) + "\n\n" + best_summary


In [35]:
# def analyze_manual_contracts(df: pd.DataFrame, top_n: int = 5, context_note: str = "Evaluating currently held option trades") -> str:
#     intro = f"""
# You are an advanced options trading assistant. These contracts are currently held positions. Evaluate each one considering:

# - Profit or loss based on fill price vs mid (if available)
# - Risk of theta decay and time to expiry
# - Breakeven realism and expected return
# - Alignment with trader goals (swing, core hold, speculative)
# - Give a clear recommendation: Hold, Adjust, or Exit
# - If the trade is profitable, note momentum potential. If it's underwater, evaluate recovery odds or risk management options.

# {context_note.strip()}

# --- Begin Trade Analysis ---
# """.strip()

#     sorted_df = df.sort_values("composite_score", ascending=False).head(top_n)
#     trade_blocks = []

#     for _, row in sorted_df.iterrows():
#         symbol = row["symbol"]
#         strike = row["strike"]
#         expiry = row["expiration"]
#         underlying = row["underlying"]
#         price = row["underlying_price"]
#         breakeven = row.get("breakeven_price", 0)
#         fill = row.get("fill_price", np.nan)
#         mid = row.get("mid", np.nan)
#         pnl = row.get("unrealized_pnl", np.nan)
#         pnl_pct = row.get("pnl_pct", np.nan)
#         days = row.get("days_to_expiry", 0)
#         roi = row.get("expected_roi", 0)
#         delta = row.get("delta", 0)
#         theta = row.get("theta", 0)
#         iv = row.get("iv", 0)
#         tags = row.get("smart_tags", "—")
#         prob_itm = row.get("prob_ITM", 0)
#         bid = row.get("bid", np.nan)
#         ask = row.get("ask", np.nan)

#         # === Recommendation Logic ===
#         rec_notes = []

#         # Handle missing data gracefully
#         has_valid_prices = not np.isnan(fill) and not np.isnan(mid)
#         pnl_info = f"{pnl:.2f} ({pnl_pct:.2f}%)" if has_valid_prices else "—"

#         if has_valid_prices:
#             if pnl_pct > 20 and roi > 0 and theta > -0.5:
#                 rec_notes.append("✅ **Hold** — Strong ROI and manageable theta decay.")
#             elif pnl_pct < -30 and days < 30 and theta < -1:
#                 rec_notes.append("⚠️ **Consider Exiting** — Underwater with time working against you.")
#             elif 0 < pnl_pct < 15 and delta > 0.3 and days > 30:
#                 rec_notes.append("🔁 **Hold or Adjust** — Modest gain and time left.")
#         else:
#             rec_notes.append("⚠️ **Data Missing** — Fill or mid price unavailable. P/L unknown.")

#         # Add strategic overlays even if P/L is missing
#         if roi > 100 and prob_itm < 0.25:
#             rec_notes.append("🎯 **Speculative Longshot** — High ROI, but low probability.")
#         if days < 10 and theta < -0.5:
#             rec_notes.append("⏳ **Theta Risk** — Decay accelerating as expiration nears.")
#         if price < breakeven * 0.8:
#             rec_notes.append("🚫 **Breakeven Far** — Low chance of profit without major move.")
#         if not np.isnan(bid) and not np.isnan(ask) and (ask - bid) > 0.5:
#             rec_notes.append("💸 **Wide Spread** — Could be costly to exit.")

#         recommendation = " ".join(rec_notes)

#         block = f"""
# ---
# 🔹 **Symbol:** {symbol}
# - Underlying: {underlying} @ ${price:.2f} | Strike: {strike} | Expiry: {expiry}
# - Fill Price: ${fill:.2f} | Mid: {mid if not np.isnan(mid) else '—'} | P/L: {pnl_info}
# - Breakeven: ${breakeven:.2f} | Days to Expiry: {days} | Expected ROI: {roi:.2f}%
# - Greeks → Delta: {delta:.2f} | Theta: {theta:.2f} | IV: {iv:.2f} | Prob ITM: {prob_itm:.2f}
# - Tags: {tags}

# 💡 Recommendation: {recommendation}
# """.strip()

#         trade_blocks.append(block)

#     return intro + "\n\n" + "\n\n".join(trade_blocks)


In [36]:
#    """Builds a prompt from a trade row for LLM assessment."""
# === LLM Trade Prompt Builder ===
def format_option_prompt(row: pd.Series, trade_type: str = "screener") -> str:
    # Safely extract with fallbacks
    symbol = row.get("symbol", "N/A")
    expiry = row.get("expiration", "N/A")
    underlying = row.get("underlying", "N/A")
    spot = row.get("underlying_price", 0)
    strike = row.get("strike", 0)
    cost = row.get("cost", 0)
    delta = row.get("delta", 0)
    theta = row.get("theta", 0)
    gamma = row.get("gamma", 0)
    iv = row.get("iv", 0)
    iv_rank = row.get("iv_rank", 0)
    roi = row.get("expected_roi", 0)
    winrate = row.get("mc_winrate", 0)
    prob_itm = row.get("prob_ITM", 0)
    breakeven = row.get("breakeven_price", strike + cost)
    target = row.get("target_price", 0)
    kelly = row.get("kelly_weight", 0)
    sharpe = row.get("sharpe_style_score", 0)
    tags = row.get("smart_tags", "—")
    composite = row.get("composite_score", 0)
    signal_score = row.get("signal_score", 0)
    fill_price = row.get("fill_price", None)
    mid = row.get("mid", None)
    pnl = row.get("unrealized_pnl", None)
    pnl_pct = row.get("pnl_pct", None)

    prompt = f"""🔹 Trade Type: {trade_type.capitalize()} | Symbol: {symbol} | Expiry: {expiry}
🔹 Underlying: {underlying} @ ${spot:.2f} | Strike: {strike} | Cost: ${cost:.2f}
🔹 Greeks → Delta: {delta:.2f}, Theta: {theta:.2f}, Gamma: {gamma:.3f}
🔹 Vol Metrics → IV: {iv:.2f} | IV Rank: {iv_rank:.2f}
🔹 Simulated → ROI: {roi:.2f}% | Winrate: {winrate:.2f} | Prob ITM: {prob_itm:.2f}
🔹 Targets → Breakeven: ${breakeven:.2f} | Target: ${target:.2f}
🔹 Risk Flags → Kelly: {kelly:.2f} | Sharpe-style Score: {sharpe:.2f}
🔹 Tags: {tags}
🏁 Composite Score: {composite:.2f} | Signal Score: {signal_score:.2f}"""

    if row.get("probabilistic_risk", False):
        prompt += "\n⚠️ Probabilistic Risk Detected"
    if row.get("conviction_flag", False):
        prompt += "\n✅ Conviction-Grade Setup"

    if fill_price is not None and mid is not None and pnl is not None:
        prompt += f"\n📊 Fill Price: ${fill_price:.2f} | Mid: ${mid:.2f} | P/L: ${pnl:.2f} ({pnl_pct:.2f}%)"

    return prompt




In [37]:
def analyze_option_trades(
    df: pd.DataFrame,
    top_n: int = 5,
    trade_type: str = "manual",
    include_llm: bool = False,
    context_note: str = "Evaluating current trades"
) -> str:
    if df.empty:
        return "⚠️ No data available for analysis."

    is_manual = trade_type.lower() == "manual"
    title = "Held Trades" if is_manual else "Top Screener Trades"
    intro = f"""
You are an advanced options trading assistant. Below is an evaluation of {title}.

Evaluation Criteria:
- Profitability (P/L, ROI, breakeven realism)
- Greeks (theta decay, delta exposure)
- Strategic signals and option-specific risks
- Recommendation: Hold, Adjust, Exit or Consider

{context_note.strip()}

--- Begin Trade Evaluation ---
""".strip()

    sorted_df = df.sort_values("composite_score", ascending=False).head(top_n)
    trade_blocks = []

    for _, row in sorted_df.iterrows():
        symbol = row["symbol"]
        underlying = row.get("underlying", "—")
        strike = row.get("strike", 0)
        expiry = row.get("expiration", "—")
        price = row.get("underlying_price", 0)
        fill = row.get("fill_price", np.nan)
        mid = row.get("mid", np.nan)
        pnl = row.get("unrealized_pnl", np.nan)
        pnl_pct = row.get("pnl_pct", np.nan)
        breakeven = row.get("breakeven_price", row.get("strike", 0) + row.get("cost", 0))
        days = row.get("days_to_expiry", 0)
        roi = row.get("expected_roi", 0)
        delta = row.get("delta", 0)
        theta = row.get("theta", 0)
        gamma = row.get("gamma", 0)
        iv = row.get("iv", 0)
        prob_itm = row.get("prob_ITM", 0)
        tags = row.get("smart_tags", "—")
        composite_score = row.get("composite_score", 0)
        signal_score = row.get("signal_score", 0)
        target = row.get("target_price", 0)

        # Recommendation logic
        rec = "🕵️ **Watch Closely** — Neutral or unclear signal."
        if not np.isnan(pnl_pct) and not np.isnan(fill) and not np.isnan(mid):
            if pnl_pct > 20 and roi > 0:
                rec = "✅ **Hold** — Good P/L and simulated ROI."
            elif pnl_pct < -30 and days < 30:
                rec = "⚠️ **Consider Exiting** — Losses + low time value."
            elif roi > 100 and prob_itm < 0.2:
                rec = "🎯 **Speculative** — High potential, low odds."
        if price < breakeven * 0.8:
            rec += " 🚫 Breakeven is far — low chance of profitability without a strong move."

        # Build block
        block = f"""
---
🔹 **Symbol:** {symbol}
- Underlying: {underlying} @ ${price:.2f} | Strike: {strike} | Expiry: {expiry}
- Fill Price: ${fill:.2f} | Mid: ${mid:.2f} | P/L: ${pnl:.2f} ({pnl_pct:.2f}%)
- Breakeven: ${breakeven:.2f} | Days to Expiry: {days} | Expected ROI: {roi:.2f}%
- Greeks → Delta: {delta:.2f} | Theta: {theta:.2f} | IV: {iv:.2f} | Prob ITM: {prob_itm:.2f}
- Tags: {tags}

💡 Recommendation: {rec}
""".strip()

        # === Optional LLM logic ===
        if include_llm:
            try:
                llm_prompt = f"""
You are a highly analytical options trading assistant. Evaluate the following options contract and explain whether it is realistically profitable, risky, or speculative — and why — using data-based reasoning.

🔹 Trade Type: {trade_type.capitalize()} | Symbol: {symbol} | Expiry: {expiry}
🔹 Underlying: {underlying} @ ${price:.2f} | Strike: {strike} | Cost: ${row.get('cost', 0):.2f}
🔹 Greeks → Delta: {delta:.2f}, Theta: {theta:.2f}, Gamma: {gamma:.3f}
🔹 Volatility → Implied Volatility (IV): {iv:.2f}
🔹 Simulations → ROI: {roi:.2f}% | Winrate: {row.get('mc_winrate', 0):.2f} | Prob ITM: {prob_itm:.2f}
🔹 Risk → Kelly: {row.get('kelly_weight', 0):.2f} | Sharpe Score: {row.get('sharpe_style_score', 0):.2f}
🔹 Pricing → Breakeven: ${breakeven:.2f} | Target: ${target:.2f}
🔹 Time → Days to Expiry: {days} | Theta Decay: {theta:.2f}
🔹 Tags: {tags}
📊 Position → Fill: ${fill:.2f} | Mid: ${mid:.2f} | P/L: ${pnl:.2f} ({pnl_pct:.2f}%) | Composite Score: {composite_score:.2f}

Please answer:
1. Is this trade speculative, conservative, or balanced? Why?
2. Based on delta, theta, and days to expiry, what is the **ideal holding timeframe** (e.g., intraday scalp, 1-2 day swing, full expiry)?
3. Are there any contradictions in the data? (e.g., high Sharpe but low Winrate — explain it.)
4. What would you **watch out for** before taking this trade?
5. Give your **verdict**: Hold, Exit, Adjust, or Consider — with reasoning.
"""
                llm_output = get_llm_insights(llm_prompt)
                block += f"\n\n🧠 **LLM Insight:**\n{llm_output.strip()}"
            except Exception as e:
                block += f"\n\n⚠️ **LLM Error:** {e}"

        trade_blocks.append(block)

    # Best pick summary
    top_trade = sorted_df.iloc[0]
    top_symbol = top_trade["symbol"]
    top_score = top_trade["composite_score"]
    top_tags = top_trade.get("smart_tags", "—")

    summary = f"""
--- Final Summary ---

🧠 You reviewed {top_n} {'held' if is_manual else 'scouted'} trades.

✅ Prioritize setups with strong ROI and delta support and give an explanation of why based on the data this is the best trade and
if it's a short or longterm hold move, intraday, day etc...
⏳ Exit or adjust if theta decay outpaces potential recovery.
🎯 **Top Opportunity:** {top_symbol} — Score: {top_score:.2f}, Tags: {top_tags}

""".strip()

    return intro + "\n\n" + "\n\n".join(trade_blocks) + "\n\n" + summary


In [38]:
LLM_MODEL = "google/gemini-2.0-flash-thinking-exp:free"  # Or any valid OpenRouter model name

def get_llm_insights(prompt: str, model: str = "google/gemini-2.0-flash-thinking-exp:free") -> str:
    import os
    import requests

    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise ValueError("❌ Missing OpenRouter API key. Set it using os.environ['OPENROUTER_API_KEY'].")

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://yourproject.com",
        "X-Title": "Option Strategy Assistant"
    }

    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


In [39]:
# === Generate Unique Cache Paths Based on Tickers ===
cache_folder = "/content/drive/MyDrive/11_Trading/cache"
ticker_hash = hashlib.md5(",".join(sorted(TICKERS)).encode()).hexdigest()[:8]

OPTIONS_CACHE_CSV = f"{cache_folder}/options_snapshot_{ticker_hash}.csv"
CACHE_META_PATH = f"{cache_folder}/meta_{ticker_hash}.json"
LLM_CACHE_PATH = f"{cache_folder}/llm_responses.json"

FORCE_LLM_REFRESH = False  # Set to True to force a reanalysis

# === Load/Save LLM Cache ===
def load_llm_cache():
    if os.path.exists(LLM_CACHE_PATH):
        with open(LLM_CACHE_PATH, "r") as f:
            return json.load(f)
    return {}

def save_llm_cache(cache):
    with open(LLM_CACHE_PATH, "w") as f:
        json.dump(cache, f, indent=2)

# === Build Cache Key Based on Symbols Being Analyzed ===
def build_llm_cache_key(top_global: pd.DataFrame) -> str:
    symbols = ",".join(sorted(top_global["symbol"].astype(str)))
    return f"top_global_response_{hashlib.md5(symbols.encode()).hexdigest()[:8]}"

# === Check if Cache Is Fresh ===
def is_llm_cache_fresh(llm_cache, key, freshness_minutes=15):
    entry = llm_cache.get(key, {})
    ts = entry.get("last_updated")
    if not ts:
        return False
    try:
        last_updated = datetime.fromisoformat(ts)
        return datetime.utcnow() - last_updated < timedelta(minutes=freshness_minutes)
    except Exception as e:
        print(f"⚠️ LLM cache timestamp error: {e}")
        return False

# === Update Timestamp for Generic Caches (Optional) ===
def update_cache_timestamp(meta_path: str):
    os.makedirs(os.path.dirname(meta_path), exist_ok=True)
    with open(meta_path, "w") as f:
        json.dump({"last_updated": datetime.utcnow().isoformat()}, f)

# === Initialize LLM Cache Dict ===
llm_cache = load_llm_cache()

In [40]:
def run_option_scoring_pipeline(final_df, top_n_per_ticker=10, top_n_global=10, strategy_override=None):
    # === Type casting and NaN prep ===
    final_df["cost"] = pd.to_numeric(final_df["cost"], errors="coerce")
    final_df["iv"] = pd.to_numeric(final_df["iv"], errors="coerce")
    final_df["expected_roi"] = pd.to_numeric(final_df["expected_roi"], errors="coerce")

    # === Strategy tagging (requires spot price) ===
    avg_spot = final_df["underlying_price"].mean()
    final_df = assign_strategy_tags(final_df, spot_price=avg_spot)

    # === Fill defaults ===
    final_df["mc_winrate"] = final_df["mc_winrate"].fillna(0).clip(0, 1)
    final_df["prob_ITM"] = final_df["prob_ITM"].fillna(0).clip(0, 1)
    final_df["expected_roi"] = final_df["expected_roi"].fillna(0)
    final_df["days_to_expiry"] = final_df["days_to_expiry"].replace(0, 1)
    final_df["delta"] = final_df["delta"].fillna(0)
    final_df["cost"] = final_df["cost"].replace(0, 1)
    final_df["strike"] = final_df["strike"].fillna(0)
    final_df["underlying_price"] = final_df["underlying_price"].fillna(final_df["strike"])

    # === Cost Buckets ===
    final_df["cost_bucket"] = pd.cut(
        final_df["cost"],
        bins=[0, 0.5, 2, 5, 20, 100],
        labels=["🟢 Ultra Cheap", "🟡 Cheap", "🟠 Moderate", "🔴 Premium", "🚫 Expensive"]
    )

    # === Derived scoring metrics ===
    final_df["score_winrate"] = final_df["mc_winrate"]
    final_df["score_prob_ITM"] = final_df["prob_ITM"]
    final_df["score_roi_efficiency"] = final_df["expected_roi"] / final_df["days_to_expiry"]
    final_df["score_delta_weighted"] = final_df["delta"] * final_df["cost"]
    final_df["breakeven_price"] = final_df["strike"] + final_df["cost"]
    final_df["breakeven_distance"] = (final_df["breakeven_price"] - final_df["underlying_price"]) / final_df["underlying_price"]
    final_df["score_breakeven_buffer"] = 1 / (1 + final_df["breakeven_distance"].clip(lower=0.01))

    final_df["kelly_weight"] = final_df["mc_winrate"] - (
        (1 - final_df["mc_winrate"]) / (final_df["expected_roi"] / 100).replace(0, np.nan)
    )
    final_df["kelly_weight"] = final_df["kelly_weight"].clip(lower=0)

    roi_std = final_df["expected_roi"].std() or 1
    final_df["sharpe_style_score"] = final_df["expected_roi"] / roi_std
    final_df["probabilistic_risk"] = final_df["sharpe_style_score"] < 0.5
    final_df["conviction_flag"] = final_df["sharpe_style_score"] > 1

    final_df["composite_score"] = (
        0.20 * final_df["score_winrate"] +
        0.20 * final_df["score_roi_efficiency"] +
        0.15 * final_df["score_breakeven_buffer"] +
        0.15 * final_df["score_prob_ITM"] +
        0.15 * final_df["score_delta_weighted"] +
        0.10 * final_df["kelly_weight"] +
        0.05 * final_df["sharpe_style_score"]
    )

    # === Supplemental indicators ===
    final_df["iv_rank"] = final_df.groupby("underlying")["iv"].rank(pct=True)
    final_df["below_max_pain"] = final_df["strike"] < final_df["max_pain"]
    final_df["within_10pct"] = (abs(final_df["strike"] - final_df["underlying_price"]) / final_df["underlying_price"]) < 0.1
    final_df["volatility_favorable"] = final_df["iv"] > 0.5
    final_df["roi_positive"] = final_df["expected_roi"] > 0
    final_df["theta_safe"] = final_df["theta"] > -1.0
    final_df["early_exit_ok"] = (final_df["expected_roi"] > 5) & (final_df["theta"] > -0.5)
    final_df["volatility_sell_ok"] = (final_df["iv_rank"] > 0.6) & (final_df["delta"] > 0.3)
    final_df["is_below_max_pain_candidate"] = (
        final_df["below_max_pain"] &
        final_df["within_10pct"] &
        final_df["volatility_favorable"] &
        final_df["roi_positive"]
    )

    # === Breakeven realism check ===
    final_df["target_distance"] = (final_df["breakeven_price"] - final_df["target_price"]) / final_df["underlying_price"]
    final_df["breakeven_realistic"] = final_df["target_price"] >= final_df["breakeven_price"]
    final_df["somewhat_realistic"] = final_df["target_distance"] < 0.05

    pre_realism = len(final_df)
    final_df = final_df[final_df["breakeven_realistic"]]
    print(f"🧮 Breakeven realism: {len(final_df)} of {pre_realism} trades kept")

    if final_df.empty:
        print("❌ No trades after breakeven realism filter.")
        return pd.DataFrame()

    # === Soft filtering ===
    final_df = final_df[
        (final_df["expected_roi"] > -10) &
        (final_df["theta"] > -3.0) &
        (final_df["iv"].notnull()) &
        (final_df["delta"].abs() <= 1.5) &
        (final_df["cost"] < 6) &
        (final_df["volume"] > 20) &
        (final_df["open_interest"] > 10)
    ]

    if final_df.empty:
        print("❌ No trades after soft filters.")
        return pd.DataFrame()

    # === Final sort & filter ===
    sorted_df = final_df.sort_values("composite_score", ascending=False)
    top_options = select_top_options_by_ticker(sorted_df, top_n=top_n_per_ticker)

    if strategy_override:
        top_options = top_options[
            top_options["smart_tags"].apply(lambda tags: any(tag in tags for tag in strategy_override))
        ]
        print(f"🎯 Strategy Filter Applied: {strategy_override}")

    top_global = top_options.sort_values("composite_score", ascending=False).head(top_n_global)

    # === Display Block ===
    cols_to_show = [col for col in [
        "symbol", "underlying", "expiration", "composite_score", "expected_roi", "iv", "cost", "cost_bucket",
        "target_distance", "theta", "volume", "open_interest",
        "early_exit_ok", "smart_tags"
    ] if col in top_global.columns]

    print("🔍 Top Trades by Composite Score:")
    display(top_global[cols_to_show])

    return top_global


In [41]:
# def run_option_scoring_pipeline(final_df, top_n_per_ticker=3, top_n_global=10, strategy_override=None):
#     # === Numeric coercion and cleaning ===
#     numeric_cols = ["ask", "bid", "volume", "open_interest", "cost", "iv"]
#     for col in numeric_cols:
#         final_df[col] = pd.to_numeric(final_df[col], errors="coerce")
#     final_df = final_df.dropna(subset=["ask", "bid", "volume", "open_interest"])
#     print(f"🔢 After dropna on pricing essentials: {len(final_df)} rows")

#     # === Check if DataFrame is empty after initial cleaning ===
#     if final_df.empty:
#         print("⚠️ No data available after initial cleaning. Exiting pipeline.")
#         return pd.DataFrame()

#     # === Simulate target_price, mc_winrate, expected_roi, prob_hit_target ===
#     print("🔄 Simulating target_price, mc_winrate, expected_roi, prob_hit_target...")
#     sim_results = final_df.apply(simulate_target_price, axis=1)
#     sim_results_df = pd.DataFrame(sim_results.tolist(), columns=["target_price", "mc_winrate", "expected_roi", "prob_hit_target"])
#     final_df = pd.concat([final_df.reset_index(drop=True), sim_results_df], axis=1)

#     # === Check if DataFrame is empty after simulation ===
#     if final_df.empty:
#         print("⚠️ No data available after simulation. Exiting pipeline.")
#         return pd.DataFrame()

#     # === Strategy Tags ===
#     avg_spot = final_df["underlying_price"].mean()
#     final_df = assign_strategy_tags(final_df, spot_price=avg_spot)

#     # === Fill defaults and fix edge cases ===
#     final_df["mc_winrate"] = final_df["mc_winrate"].fillna(0).clip(0, 1)
#     final_df["prob_ITM"] = final_df.get("prob_ITM", pd.Series(0)).fillna(0).clip(0, 1)
#     final_df["days_to_expiry"] = final_df["days_to_expiry"].replace(0, 1)
#     final_df["delta"] = final_df["delta"].fillna(0)
#     final_df["cost"] = final_df["cost"].replace(0, 1)
#     final_df["strike"] = final_df["strike"].fillna(0)
#     final_df["underlying_price"] = final_df["underlying_price"].fillna(final_df["strike"])

#     # === Derived & scoring metrics ===
#     final_df["score_winrate"] = final_df["mc_winrate"]
#     final_df["score_prob_ITM"] = final_df["prob_ITM"]
#     final_df["score_roi_efficiency"] = final_df["expected_roi"] / final_df["days_to_expiry"]
#     final_df["score_delta_weighted"] = final_df["delta"] * final_df["cost"]
#     final_df["breakeven_price"] = final_df["strike"] + final_df["cost"]
#     final_df["breakeven_distance"] = (final_df["breakeven_price"] - final_df["underlying_price"]) / final_df["underlying_price"]
#     final_df["score_breakeven_buffer"] = 1 / (1 + final_df["breakeven_distance"].clip(lower=0.01))

#     final_df["kelly_weight"] = final_df["mc_winrate"] - (
#         (1 - final_df["mc_winrate"]) / (final_df["expected_roi"] / 100).replace(0, np.nan)
#     )
#     final_df["kelly_weight"] = final_df["kelly_weight"].clip(lower=0)

#     roi_std = final_df["expected_roi"].std() or 1
#     final_df["sharpe_style_score"] = final_df["expected_roi"] / roi_std
#     final_df["probabilistic_risk"] = final_df["sharpe_style_score"] < 0.5
#     final_df["conviction_flag"] = final_df["sharpe_style_score"] > 1

#     final_df["composite_score"] = (
#         0.20 * final_df["score_winrate"] +
#         0.20 * final_df["score_roi_efficiency"] +
#         0.15 * final_df["score_breakeven_buffer"] +
#         0.15 * final_df["score_prob_ITM"] +
#         0.15 * final_df["score_delta_weighted"] +
#         0.10 * final_df["kelly_weight"] +
#         0.05 * final_df["sharpe_style_score"]
#     )

#     # === Breakeven realism ===
#     if final_df["target_price"].notnull().any():
#         final_df["target_distance"] = (final_df["breakeven_price"] - final_df["target_price"]) / final_df["underlying_price"]
#         final_df["breakeven_realistic"] = final_df["target_price"] >= final_df["breakeven_price"]
#         pre = len(final_df)
#         final_df = final_df[final_df["breakeven_realistic"]]
#         print(f"🧮 Breakeven realism: {len(final_df)} of {pre} rows survived")
#     else:
#         print("⚠️ No valid target_price — skipping breakeven realism filtering.")
#         final_df["target_distance"] = np.nan

#     if final_df.empty:
#         print("⚠️ Nothing left after breakeven realism.")
#         return pd.DataFrame()

#     # === Spread and liquidity ===
#     final_df["spread"] = final_df["ask"] - final_df["bid"]
#     final_df["mid"] = (final_df["ask"] + final_df["bid"]) / 2
#     final_df["liquidity_score"] = (
#         (final_df["volume"] + 1).apply(np.log1p) +
#         (final_df["open_interest"] + 1).apply(np.log1p) +
#         (1 / (final_df["spread"] + 0.01))
#     )

#     # === Soft Filtering ===
#     pre_soft = len(final_df)
#     final_df = final_df[(
#         (final_df["expected_roi"] > -10) &
#         (final_df.get("theta", pd.Series(0)) > -3.0) &
#         (final_df["iv"].notnull()) &
#         (final_df["delta"].abs() <= 1.5) &
#         (final_df["cost"] < 100) &
#         (final_df["volume"] > 20) &
#         (final_df["open_interest"] > 0)
#     )]
#     print(f"🧼 Soft filters: {len(final_df)} of {pre_soft} rows survived")

#     if final_df.empty:
#         print("❌ All trades filtered out after soft filters.")
#         return pd.DataFrame()

#     # === Tags & Labels ===
#     final_df["iv_rank"] = final_df.groupby("underlying")["iv"].rank(pct=True)
#     final_df["below_max_pain"] = final_df["strike"] < final_df["max_pain"]
#     final_df["within_10pct"] = (
#         abs(final_df["strike"] - final_df["underlying_price"]) / final_df["underlying_price"]
#     ) < 0.1
#     final_df["volatility_favorable"] = final_df["iv_rank"] > 0.6
#     final_df["roi_positive"] = final_df["expected_roi"] > 0
#     final_df["theta_safe"] = final_df["theta"] > -1.0
#     final_df["early_exit_ok"] = (final_df["expected_roi"] > 5) & (final_df["theta"] > -0.5)
#     final_df["volatility_sell_ok"] = (final_df["iv_rank"] > 0.6) & (final_df["delta"] > 0.3)
#     final_df["is_below_max_pain_candidate"] = (
#         final_df["below_max_pain"] &
#         final_df["within_10pct"] &
#         final_df["volatility_favorable"] &
#         final_df["roi_positive"]
#     )

#     # === Final Selection ===
#     sorted_df = final_df.sort_values("composite_score", ascending=False)
#     top_options = select_top_options_by_ticker(sorted_df, top_n=top_n_per_ticker)

#     if strategy_override:
#         top_options = top_options[
#             top_options["smart_tags"].apply(lambda tags: any(tag in tags for tag in strategy_override))
#         ]
#         print(f"🎯 Strategy Filter Applied: {strategy_override}")

#     top_global = top_options.sort_values("composite_score", ascending=False).head(top_n_global)

#     print("🔍 Top Trades by Composite Score:")
#     display(top_global[[
#         "symbol", "composite_score", "expected_roi", "iv", "iv_rank",
#         "target_distance", "theta", "gamma", "volume", "open_interest",
#         "early_exit_ok", "volatility_sell_ok", "smart_tags"
#     ]])

#     return top_global


In [42]:
# === Fetch all available expiration dates for a given ticker ===
def get_all_expirations(ticker: str):
    try:
        contracts = client.list_snapshot_options_chain(ticker)
        expirations = list(set(getattr(contract.details, "expiration_date", None) for contract in contracts if contract.details))
        expirations = [e for e in expirations if e]  # remove None values
        expirations = sorted(set(pd.to_datetime(e).strftime("%Y-%m-%d") for e in expirations))
        return expirations
    except Exception as e:
        print(f"❌ Error fetching expirations for {ticker}: {e}")
        return []

In [43]:
def plot_option_payoff_heatmap(row, price_range_pct=None, time_steps=15, mode="percent", show=True):
    """
    Enhanced payoff heatmap with:
    - Spot price overlay
    - Breakeven overlay
    - Target price path overlay
    - Max Pain overlay
    - Clean legend and intuitive layout
    """
    from datetime import datetime, timedelta
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    strike = row["strike"]
    cost = row["cost"]
    expiry = pd.to_datetime(row["expiration"])
    spot = row["underlying_price"]
    days_left = int(row["days_to_expiry"])

    if pd.isna(spot) or pd.isna(cost) or days_left <= 0:
        print(f"⚠ Skipping {row['symbol']} — invalid spot, cost, or expiry.")
        return

    # === Price Range Logic ===
    if price_range_pct is None:
        distance_pct = abs(spot - strike) / spot
        price_range_pct = max(0.2, distance_pct + 0.1)

    price_low = min(strike, spot) * (1 - price_range_pct)
    price_high = max(strike, spot) * (1 + price_range_pct)
    price_steps = np.linspace(price_low, price_high, 30)

    # === Time Steps (Dates) ===
    day_steps = np.linspace(0, days_left, time_steps)
    date_labels = [
        (datetime.today() + timedelta(days=int(d))).strftime("%m-%d") for d in day_steps
    ]

    # === P/L Matrix ===
    pl_matrix = np.zeros((len(price_steps), len(day_steps)))
    for i, price in enumerate(price_steps):
        for j, dte in enumerate(day_steps):
            decay = dte / days_left  # 1 = now, 0 = expiry
            intrinsic = max(price - strike, 0)
            decay_value = intrinsic * decay
            pnl = decay_value - cost
            pl_matrix[i, j] = (pnl / cost * 100) if mode == "percent" else pnl

    # === Plot Heatmap ===
    plt.figure(figsize=(12, 7))
    ax = sns.heatmap(
        pl_matrix,
        cmap="RdYlGn",
        center=0,
        xticklabels=date_labels,
        yticklabels=[f"${p:.2f}" for p in price_steps],
        cbar_kws={"label": "P/L (%)" if mode == "percent" else "P/L ($)"}
    )

    # === Overlays ===

    # Spot price
    spot_idx = np.argmin(np.abs(price_steps - spot))
    ax.hlines(spot_idx + 0.5, *ax.get_xlim(), colors="blue", linestyles="--", linewidth=1.5, label="📍 Spot Price")

    # Breakeven price
    breakeven = strike + cost
    breakeven_idx = np.argmin(np.abs(price_steps - breakeven))
    ax.hlines(breakeven_idx + 0.5, *ax.get_xlim(), colors="orange", linestyles="--", linewidth=1.5, label="🧮 Breakeven")

    # Target price path
    if "target_price" in row and pd.notna(row["target_price"]):
        target = row["target_price"]
        target_prices = np.linspace(spot, target, len(day_steps))
        target_idx = [np.argmin(np.abs(price_steps - p)) + 0.5 for p in target_prices]
        ax.plot(range(len(day_steps)), target_idx, color="purple", linestyle="-.", linewidth=2, label="🎯 Target Path")

    # Max Pain overlay
    if "max_pain" in row and pd.notna(row["max_pain"]):
        max_pain = row["max_pain"]
        max_pain_idx = np.argmin(np.abs(price_steps - max_pain))
        ax.hlines(max_pain_idx + 0.5, *ax.get_xlim(), colors="red", linestyles=":", linewidth=1.5, label="📉 Max Pain")

    # === Final Layout ===
    ax.set_title(f"Payoff Heatmap — {row['symbol']} | Strike: {strike} | Cost: ${cost:.2f}")
    ax.set_xlabel("Simulated Date")
    ax.set_ylabel("Stock Price")
    ax.invert_yaxis()

    # === Clean Legend (deduplicated)
    handles, labels = ax.get_legend_handles_labels()
    unique = dict(zip(labels, handles))
    ax.legend(unique.values(), unique.keys(), title="Overlays", loc="upper left", bbox_to_anchor=(1.17, 1))

    plt.tight_layout()
    if show:
        plt.show()
    return plt.gcf()  # return the current figure


In [44]:
print(llm_cache)

{}


In [45]:
def run_llm_batch_analysis(df: pd.DataFrame, trade_type: str = "screener"):
    for _, row in df.iterrows():
        print(f"\n📈 {trade_type.capitalize()} Trade: {row['symbol']}")
        prompt = format_option_prompt(row, trade_type=trade_type)
        print(get_llm_insights(prompt))


In [ ]:
# === STATS AND SNAPSHOT PIPELINE EXECUTION ===

# Step 1: Fetch Spot Prices
spot_dict = get_spot_price(tickers)
all_snapshots = []
#max_expiry_days = 60  # ⏳ look up to 60 days ahead (adjust as desired)
# Define the minimum and maximum expiry days
min_expiry_days = 30
max_expiry_days = 365


today = datetime.today()

for t in tickers:
    try:
        spot = spot_dict.get(t, np.nan)
        if pd.isna(spot):
            print(f"⚠ Skipping {t} — Spot price missing.")
            continue

        # Fetch all expiration dates for the ticker
        all_expiries = get_all_expirations(t)  # Ensure this function is defined

        # Filter expirations between min_expiry_days and max_expiry_days from today
        filtered_expiries = [
            expiry for expiry in all_expiries
            if min_expiry_days <= (pd.to_datetime(expiry) - today).days <= max_expiry_days
        ]

        for expiry in filtered_expiries:
            df, _ = fetch_option_chain_snapshots(t, contract_type="call", expiry=expiry)
            if df.empty:
                continue

            df = enrich_polygon_df(df, spot)
            df["underlying_price"] = spot
            df["spot_price"] = spot
            df["days_to_expiry"] = (pd.to_datetime(expiry) - today).days

            max_pain_strike = get_max_pain(t, expiry, spot_dict)
            df["max_pain"] = max_pain_strike
            df["distance_to_max_pain"] = abs(df["strike"] - max_pain_strike) / spot

            all_snapshots.append(df)

            print(f"✅ {t} | {expiry} | Contracts: {len(df)} | Max Pain: {max_pain_strike}")

    except Exception as e:
        print(f"❌ Error processing {t}: {e}")

# Step 3: Concatenate and snapshot raw unfiltered data
final_df = pd.concat(all_snapshots, ignore_index=True) if all_snapshots else pd.DataFrame()
exp_df = final_df.copy()

# Step 4: Show full snapshot of all pulled contracts BEFORE filtering
print("🧾 Full Snapshot of All Contracts (Pre-Filtering):")
print(f"Shape: {final_df.shape}")
display(final_df[["symbol", "underlying", "strike", "expiration", "ask", "bid", "mid", "spot_price", "iv"]].head(25))

In [ ]:
# Step 5: Validate key columns before scoring
missing_values = final_df[['spot_price', 'underlying_price']].isna().sum()
if missing_values.any():
    print(f"⚠️ Missing values: {missing_values}")
else:
    print("✅ All key pricing columns present.")

# Step 6: Score and filter
if not final_df.empty:
    final_df = run_option_scoring_pipeline(final_df)
    print(f"✅ Composite score calculated successfully.")
    print(f"🔍 Final (Scored) Shape: {final_df.shape}")
else:
    print("⚠️ final_df is empty. Skipping scoring pipeline.")


In [ ]:
print(f"Shape after enrichment: {final_df.shape}")
print(final_df.head())

In [ ]:
print(f"Shape after enrichment: {df.shape}")
print(df.head())


In [ ]:
required_columns = ['underlying_price', 'strike', 'cost', 'target_price']
missing_data = final_df[required_columns].isnull().sum()
if missing_data.any():
    print(f"⚠️ Missing data detected in columns:\n{missing_data}")
    # Handle missing data appropriately, e.g., fill with default values or drop rows
else:
    print("✅ All required columns are present and filled.")


In [ ]:
# Compute the average spot price
avg_spot = final_df["underlying_price"].mean()

# Check if avg_spot is a valid number
if pd.notna(avg_spot):
    print(f"Average spot price calculated: {avg_spot}")
    print(f"Shape before assign_strategy_tags: {final_df.shape}")
    final_df = assign_strategy_tags(final_df, spot_price=avg_spot)
    print(f"Shape after assign_strategy_tags: {final_df.shape}")
else:
    print("⚠️ Unable to calculate avg_spot due to missing or invalid data in 'underlying_price' column.")


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
display(final_df)

In [ ]:
# Show the 'code' and 'smart_tags' columns for all contracts
pd.set_option('display.max_columns', None)
display(final_df[['symbol','smart_tags']])


In [ ]:
# Print full smart_tags without truncation
for i, row in final_df.iterrows():
    print(f"{row['symbol']} — {row['smart_tags']}")


In [ ]:
figs = []
top_options = final_df.head(10)

for _, row in top_options.iterrows():
    fig = plot_option_payoff_heatmap(row, show=False)
    figs.append(fig)


In [ ]:
from pathlib import Path

out_dir = Path(r"H:\My Drive\11_Trading\Heatmaps")
out_dir.mkdir(parents=True, exist_ok=True)  # ensure it exists, create if needed


In [ ]:

for i, fig in enumerate(figs, start=1):
    fig.savefig(out_dir / f"{i:02d}_payoff.png", dpi=300, bbox_inches="tight")
    plt.close(fig)


In [ ]:
import os

print("Exists?", os.path.exists("figures"))
print("Is file?", os.path.isfile("figures"))
print("Is dir?", os.path.isdir("figures"))


In [ ]:
# === LLM Summary of Filtered Top Scored Trades ===
# NOTE: final_df already contains only the top_n_global results
llm_top_trades_summary = analyze_option_trades(
    df=final_df,
    top_n=2,  # You can change this to len(final_df) to analyze all filtered
    trade_type="screener",
    include_llm=True,
    context_note="Final filtered and enriched trades from the core scoring pipeline. Evaluate directional alignment, volatility edge, and entry risk."
)

# Print LLM feedback
print(llm_top_trades_summary)


In [ ]:

print(exp_df)


In [ ]:
manual_df = fetch_manual_contracts(my_contracts)

manual_analysis = analyze_option_trades(
    manual_df,
    top_n=2,
    trade_type="manual",
    include_llm=True,
    context_note="LLM review of live filled trades with scoring and risk context."
)
print(manual_analysis)


In [ ]:
manual_df.info()


In [ ]:
# === Step 3.5: Score full dataset without filtering ===
exp_df = compute_composite_score(exp_df)
# Ensure all columns are visible in output (especially in Colab)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Now apply the relaxed filters on the already copied dataframe (exp_df)
further_from_max_pain = exp_df[exp_df["distance_to_max_pain"] > DISTANCE_THRESHOLD] # Changed exp_filter to exp_df
print(f"Number of trades after distance filter: {len(further_from_max_pain)}")

# Apply composite score, volume, and distance filters for highlighting (experiment stage)
highlighted_options = further_from_max_pain[
    (further_from_max_pain["volume"] > VOLUME_THRESHOLD) &
    (further_from_max_pain["composite_score"] > COMPOSITE_SCORE_THRESHOLD)
]
print(f"Number of trades after volume/composite score filter: {len(highlighted_options)}")
# === Preview Top 10 Scored Trades (Unfiltered) ===
top_10_by_score = exp_df.sort_values("composite_score", ascending=False).head(10)
print("🏆 Top 10 Trades by Composite Score (Unfiltered):")
display(top_10_by_score)
# === Plotting: All trades vs. Highlighted Trades ===
# Plot all trades first (using exp_df for All Trades) # Changed exp_filter to exp_df
plt.figure(figsize=(10, 6))
sns.scatterplot(data=exp_df, x="strike", y="distance_to_max_pain", alpha=0.5, color='blue', label='All Trades')

# Highlight filtered options (Highlighted Trades = the ones meeting the relaxed filter criteria)
sns.scatterplot(data=highlighted_options, x="strike", y="distance_to_max_pain", alpha=0.7, color='red', label='Highlighted Trades')

# Add labels and titles
plt.title("Options Further from Max Pain (All Trades and Highlighted Trades)")
plt.xlabel("Strike Price")
plt.ylabel("Distance to Max Pain (Normalized)")
plt.grid(True)

# Show legend
plt.legend()

# Tight layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
#print(f"Number of trades after screening: {len(top_trades)}")


In [ ]:
# Define a threshold for ROI if not defined already
ROI_THRESHOLD = 50  # Example threshold, you can adjust this

# Ensure expected_roi is a valid column
if 'expected_roi' not in final_df.columns:
    print("⚠️ 'expected_roi' column not found!")
else:
    # Plot the histogram
    plt.figure(figsize=(10, 4))
    final_df["expected_roi"].hist(bins=50, color='skyblue', edgecolor='black')

    # Add a vertical line for ROI Threshold
    plt.axvline(ROI_THRESHOLD, color='red', linestyle='--', label=f'ROI Threshold: {ROI_THRESHOLD}%')

    # Title and labels
    plt.title("Expected ROI Distribution")
    plt.xlabel("Expected ROI (%)")
    plt.ylabel("Number of Contracts")
    plt.legend()

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
# 🔁 Visualize Top LLM Trades (Payoff Grid)
print("🎯 Visualizing Top Trades...")
for _, row in top_global.iterrows():
    plot_option_payoff_heatmap(row)


In [ ]:
# === Filter for options where distance to max pain is greater than the threshold ===
print(f"Number of trades before distance filter: {len(final_df)}")
further_from_max_pain = final_df[final_df["distance_to_max_pain"] > DISTANCE_THRESHOLD]
print(f"Number of trades after distance filter: {len(further_from_max_pain)}")

# === Filter by volume and composite score ===
print(f"Number of trades before volume/composite score filter: {len(further_from_max_pain)}")
filtered_options = further_from_max_pain[(
    further_from_max_pain["volume"] > VOLUME_THRESHOLD) &
    (further_from_max_pain["composite_score"] > COMPOSITE_SCORE_THRESHOLD)
]
print(f"Number of trades after volume/composite score filter: {len(filtered_options)}")

# === Sort the options by distance to max pain and composite score ===
sorted_filtered_options = filtered_options.sort_values(
    by=["distance_to_max_pain", "composite_score"],
    ascending=[False, False]
)
print(f"Number of trades after sorting: {len(sorted_filtered_options)}")

# Ensure "smart_tags" exists in the columns before trying to display it
columns_to_display = ["symbol", "strike", "expiration", "distance_to_max_pain", "cost", "underlying_price", "volume", "composite_score"]
if "smart_tags" in sorted_filtered_options.columns:
    columns_to_display.append("smart_tags")

# Display top 10 options further from max pain, with volume and composite score
print("🔍 Top Options Further from Max Pain (Ordered by Distance and Composite Score):")
display(sorted_filtered_options[columns_to_display].head(10))


In [ ]:
# Relaxed filters to allow more trades through
DISTANCE_THRESHOLD = 0.05  # Increase to 5% distance to max pain
VOLUME_THRESHOLD = 50  # Allow trades with lower volume (e.g., 50 contracts)
COMPOSITE_SCORE_THRESHOLD = 30  # Lower composite score threshold to include more trades

# Filter for options where distance to max pain is greater than the threshold
further_from_max_pain = final_df[final_df["distance_to_max_pain"] > DISTANCE_THRESHOLD]

# Apply composite score, volume, and distance filters
filtered_options = further_from_max_pain[(
    further_from_max_pain["volume"] > VOLUME_THRESHOLD) &
    (further_from_max_pain["composite_score"] > COMPOSITE_SCORE_THRESHOLD)
]

# Sort the options by distance to max pain and composite score
sorted_filtered_options = filtered_options.sort_values(
    by=["distance_to_max_pain", "composite_score"],
    ascending=[False, False]
)

# Ensure "smart_tags" exists in the columns before trying to display it
columns_to_display = ["symbol", "strike", "expiration", "distance_to_max_pain", "cost", "underlying_price", "volume", "composite_score"]
if "smart_tags" in sorted_filtered_options.columns:
    columns_to_display.append("smart_tags")

# Display top 10 options further from max pain, with volume and composite score
print("🔍 Top Options Further from Max Pain (Ordered by Distance and Composite Score):")
display(sorted_filtered_options[columns_to_display].head(10))

# Optionally, visualize them with a scatter plot
plt.figure(figsize=(10, 6))

# Check if composite_score exists, and use it for hue; fallback to volume if it's missing
hue = "composite_score" if "composite_score" in sorted_filtered_options.columns else "volume"

sns.scatterplot(data=sorted_filtered_options, x="strike", y="distance_to_max_pain", alpha=0.7, hue=hue, palette="viridis")
plt.title("Options Further from Max Pain (with Volume and Composite Score)")
plt.xlabel("Strike Price")
plt.ylabel("Distance to Max Pain (Normalized)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define thresholds for distance from max pain, volume, and composite score
DISTANCE_THRESHOLD = 0.02  # 2% away from max pain
VOLUME_THRESHOLD = 100  # Minimum volume threshold (example: 100 contracts)
COMPOSITE_SCORE_THRESHOLD = 50  # Minimum composite score (example: 50)

# Check if 'composite_score' column exists and calculate it if not
if 'composite_score' not in final_df.columns:
    print("⚠️ Composite score not found! Calculating it now.")
    # Make sure you have a function to calculate composite_score if it's not available
    # For now, I'm calling the calculate_composite_score function, ensure it's defined earlier
    final_df = calculate_composite_score(final_df)
    print("✅ Composite score calculated.")
else:
    print("✅ Composite score found.")

# Filter for options where distance to max pain is greater than the threshold
further_from_max_pain = final_df[final_df["distance_to_max_pain"] > DISTANCE_THRESHOLD]

# If composite score is available, filter by composite score, volume, and distance
filtered_options = further_from_max_pain[
    (further_from_max_pain["volume"] > VOLUME_THRESHOLD) &
    (further_from_max_pain["composite_score"] > COMPOSITE_SCORE_THRESHOLD)
]

# Sort the options by distance to max pain and composite score
sorted_filtered_options = filtered_options.sort_values(
    by=["distance_to_max_pain", "composite_score"],
    ascending=[False, False]
)

# Ensure "smart_tags" exists in the columns before trying to display it
columns_to_display = [
    "symbol", "strike", "expiration", "distance_to_max_pain",
    "cost", "underlying_price", "volume", "composite_score", "smart_tags"
]

# Display top 10 options further from max pain, with volume and composite score
print("🔍 Top Options Further from Max Pain (Ordered by Distance and Composite Score):")
display(sorted_filtered_options[columns_to_display].head(10))

# Optionally, visualize them with a scatter plot
plt.figure(figsize=(10, 6))

# Check if composite_score exists, and use it for hue; fallback to volume if it's missing
hue = "composite_score" if "composite_score" in sorted_filtered_options.columns else "volume"

sns.scatterplot(
    data=sorted_filtered_options,
    x="strike",
    y="distance_to_max_pain",
    alpha=0.7,
    hue=hue,
    palette="viridis"
)

plt.title("Options Further from Max Pain (with Volume and Composite Score)")
plt.xlabel("Strike Price")
plt.ylabel("Distance to Max Pain (Normalized)")
plt.grid(True)
plt.tight_layout()
plt.show()
